In [ ]:
!pip install -U bitsandbytes
!pip install -q transformers accelerate torch torchvision safetensors

In [1]:
import os
import gc

In [ ]:
from transformers import AutoProcessor, Gemma3nForConditionalGeneration, BitsAndBytesConfig
from PIL import Image
import torch
import ipywidgets as widgets
from IPython.display import display
from huggingface_hub import login
from google.colab import files
from huggingface_hub import notebook_login

In [ ]:
torch.cuda.empty_cache()
gc.collect()

430

In [ ]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [ ]:
notebook_login()

In [ ]:
torch.cuda.empty_cache()
gc.collect()

0

In [ ]:
model_id = "unsloth/gemma-3n-E2B-it-unsloth-bnb-4bit"

In [ ]:
try:

    print("Loading pre-quantized 4-bit model...")
    model = Gemma3nForConditionalGeneration.from_pretrained(
        model_id,
        device_map="auto",
        torch_dtype=torch.bfloat16,
        low_cpu_mem_usage=True,
        trust_remote_code=True
    ).eval()
    processor = AutoProcessor.from_pretrained("google/gemma-3n-e2b-it")  # Use original processor

except Exception as e:
    print(f"Pre-quantized model failed: {e}")
    print("Falling back to quantizing the original model...")

    model_id = "google/gemma-3n-e2b-it"

    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4"
    )

    model = Gemma3nForConditionalGeneration.from_pretrained(
        model_id,
        device_map="auto",
        torch_dtype=torch.bfloat16,
        low_cpu_mem_usage=True,
        trust_remote_code=True,
        quantization_config=quantization_config
    ).eval()

    processor = AutoProcessor.from_pretrained(model_id)

Loading pre-quantized 4-bit model...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
torch.cuda.empty_cache()
gc.collect()

print("Model loaded successfully!")

Model loaded successfully!


In [ ]:
def analyze_medicine_image(image_path):
    torch.cuda.empty_cache()
    gc.collect()

    image = Image.open(image_path).convert("RGB")
    # Resize to a smaller resolution to save memory (Gemma 3n supports 256x256, 512x512, 768x768)
    image = image.resize((512, 512), Image.Resampling.LANCZOS)

    messages = [
        {
            "role": "system",
            "content": [{"type": "text", "text": "You are a virtual pharmacist with deep knowledge of medicine. Based on the uploaded image of the medicine package or strip, help the user understand the medicine name and salt composition, its usage/what condition it treats, side effects, interactions with other drugs, and any warnings or precautions. Respond like a medical professional giving reliable and clear advice."}]
        },
        {
            "role": "user",
            "content": [
                {"type": "image", "image": image},
                {"type": "text", "text": "Tell me about this medicine from the image. Please provide detailed information about its name, composition, usage, side effects, and any important warnings."}
            ]
        }
    ]

    try:
        inputs = processor.apply_chat_template(
            messages,
            add_generation_prompt=True,
            tokenize=True,
            return_dict=True,
            return_tensors="pt",
        )

        inputs = {k: v.to(model.device) for k, v in inputs.items()}

        input_len = inputs["input_ids"].shape[-1]

        with torch.inference_mode():
            generation = model.generate(
                **inputs,
                max_new_tokens=512,
                do_sample=True,
                temperature=0.7,
                pad_token_id=processor.tokenizer.eos_token_id,
                use_cache=True
            )
            generation = generation[0][input_len:]
            decoded = processor.decode(generation, skip_special_tokens=True)

        del inputs, generation
        torch.cuda.empty_cache()
        gc.collect()

        return decoded

    except Exception as e:
        torch.cuda.empty_cache()
        gc.collect()
        raise e

In [ ]:
def start_conversation_loop(initial_analysis):
    print(f"🤖 Initial Analysis:\n{initial_analysis}\n")
    print("💬 Virtual Pharmacist is ready. Ask follow-up questions (type 'exit' to stop):\n")
    print("Note: Follow-up questions will be text-only based on the initial analysis.\n")

    conversation_history = [
        {
            "role": "system",
            "content": [{"type": "text", "text": f"You are a virtual pharmacist. You have already analyzed a medicine image and provided this analysis: {initial_analysis}. Answer follow-up questions based on that analysis and your medical knowledge. Keep responses concise and helpful."}]
        }
    ]

    while True:
        user_input = input("🧑‍⚕️ You: ")
        if user_input.lower() in ["exit", "quit"]:
            print("🔚 Conversation ended.")
            break

        conversation_history.append({
            "role": "user",
            "content": [{"type": "text", "text": user_input}]
        })

        try:
            torch.cuda.empty_cache()
            gc.collect()


            inputs = processor.apply_chat_template(
                conversation_history,
                add_generation_prompt=True,
                tokenize=True,
                return_dict=True,
                return_tensors="pt",
            )

            inputs = {k: v.to(model.device) for k, v in inputs.items()}

            input_len = inputs["input_ids"].shape[-1]

            with torch.inference_mode():
                generation = model.generate(
                    **inputs,
                    max_new_tokens=256,
                    do_sample=True,
                    temperature=0.7,
                    pad_token_id=processor.tokenizer.eos_token_id,
                    use_cache=True
                )
                generation = generation[0][input_len:]
                response = processor.decode(generation, skip_special_tokens=True)

            print(f"🤖 Pharmacist: {response}")

            # Add assistant response to conversation history
            conversation_history.append({
                "role": "assistant",
                "content": [{"type": "text", "text": response}]
            })

            # Clear memory after generation
            del inputs, generation
            torch.cuda.empty_cache()
            gc.collect()

        except Exception as e:
            print(f"🤖 Pharmacist: I apologize, but I'm having trouble processing your question. Please try rephrasing it or consult with a healthcare professional for additional information.")
            print(f"Error details: {str(e)}")
            # Clear memory on error
            torch.cuda.empty_cache()
            gc.collect()

In [ ]:
image_path = "/content/meds.png"

In [ ]:
initial_analysis = analyze_medicine_image(image_path)
start_conversation_loop(initial_analysis)

🤖 Initial Analysis:
## Dol-P Tablets - A Detailed Overview

Based on the image provided, the medicine is **Dol-P Tablets**. It contains a combination of two active ingredients:

**1. Diclofenac Potassium:** This is a nonsteroidal anti-inflammatory drug (NSAID). It's commonly used to relieve pain and inflammation. 

**2. Paracetamol:** This is a common analgesic (pain reliever) and antipyretic (fever reducer). It works by reducing pain and fever.

**Usage:** Dol-P Tablets are indicated for the relief of pain and inflammation associated with various conditions. These can include:

* **Musculoskeletal pain:** such as back pain, joint pain (osteoarthritis), muscle aches.
* **Inflammation:** associated with arthritis, sprains, strains, etc.
* **Mild to moderate pain:** headaches, menstrual pain, etc.

**Side Effects:** As with any medication, Dol-P can have side effects. These may include:

* **Common:** Nausea, vomiting, stomach pain, heartburn, constipation, diarrhea.
* **Less common:** S